# Snowflake Customer Insights with Cortex LLMs

This notebook replicates the data loading and modeling process for generating customer insights using Snowflake and Cortex LLM functions. It covers:
1. Setup of database, schemas, and roles.
2. Creation of file formats and stages for data ingestion.
3. Loading raw JSON data into transient tables.
4. Transforming raw data into staging tables.
5. Creating fact tables enriched with Cortex LLM functions (sentiment analysis, translation, classification, completion).
6. Building analytical tables for sentiment trends, ticket patterns, and customer summaries.
7. Developing a customer persona and signals table.
8. Running analytical queries to derive insights.
9. Adding comments to tables and columns for documentation.

## 1. Initial Setup: Roles and Warehouse
Ensure the correct role (ACCOUNTADMIN for initial setup, then DBT_ROLE) and an active warehouse (CORTEX_WH) are being used.

In [ ]:
-- Create database and schemas
USE ROLE ACCOUNTADMIN;

CREATE ROLE IF NOT EXISTS DBT_ROLE;
GRANT ROLE DBT_ROLE TO USER CURRENT_USER();

USE ROLE DBT_ROLE;

CREATE WAREHOUSE IF NOT EXISTS CORTEX_WH 
SIZE = 'X-SMALL' 
AUTO_SUSPEND = 60
AUTO_RESUME = TRUE;

USE WAREHOUSE CORTEX_WH;

DROP DATABASE IF EXISTS DBT_CORTEX_LLMS;

USE ROLE DBT_ROLE;
CREATE DATABASE IF NOT EXISTS DBT_CORTEX_LLMS;
CREATE SCHEMA IF NOT EXISTS DBT_CORTEX_LLMS.RAW;
CREATE SCHEMA IF NOT EXISTS DBT_CORTEX_LLMS.ANALYTICS;
CREATE SCHEMA IF NOT EXISTS DBT_CORTEX_LLMS.SEMANTIC_MODELS;

--This SQL Script mimics the dbt project for loading and modelling the data in Snowflake
-- Set the role for this session
USE DATABASE DBT_CORTEX_LLMS;

## 2. Database and Schema Creation
Set up the database `DBT_CORTEX_LLMS` and the necessary schemas: `RAW`, `ANALYTICS`, and `SEMANTIC_MODELS`. This structure helps organize the data flow from raw ingestion to analytical and semantic layers.

In [ ]:
DROP DATABASE IF EXISTS DBT_CORTEX_LLMS;

USE ROLE DBT_ROLE; -- Switched to DBT_ROLE as per the script's logic for creating these objects
CREATE DATABASE IF NOT EXISTS DBT_CORTEX_LLMS;
USE DATABASE DBT_CORTEX_LLMS;

CREATE SCHEMA IF NOT EXISTS DBT_CORTEX_LLMS.RAW;
CREATE SCHEMA IF NOT EXISTS DBT_Cortex_LLMS.ANALYTICS;
CREATE SCHEMA IF NOT EXISTS DBT_CORTEX_LLMS.SEMANTIC_MODELS;

## 3. File Format and Stage Setup
Create a JSON file format to handle the structure of the incoming data files. Also, set up a stage where the raw data files will be uploaded.

In [ ]:
USE SCHEMA RAW; -- Ensuring commands are executed in the RAW schema context

CREATE OR REPLACE FILE FORMAT JSON_FORMAT
    TYPE = 'JSON'
    STRIP_OUTER_ARRAY = TRUE
    COMPRESSION = 'AUTO';

CREATE OR REPLACE STAGE RAW_DATA_STAGE;

## 4. Data Upload (Placeholder)
The `PUT` commands are used to upload local files to the Snowflake stage. These commands are typically run via SnowSQL or a Snowflake connector, not directly in a SQL worksheet if the files aren't accessible from the Snowflake environment directly. 

**Note:** For this notebook, these `PUT` commands are placeholders. You would need to execute these using SnowSQL or an equivalent tool, pointing to the actual location of your JSON data files (`customer_interactions.json`, `product_reviews.json`, `support_tickets.json`, `customers.json`).

```sql
-- PUT file://data/customer_interactions.json @RAW.RAW_DATA_STAGE;
-- PUT file://data/product_reviews.json @RAW.RAW_DATA_STAGE;
-- PUT file://data/support_tickets.json @RAW.RAW_DATA_STAGE;
-- PUT file://data/customers.json @RAW.RAW_DATA_STAGE;
```

## 5. Raw Data Table Creation
Create transient tables in the `RAW` schema to initially load the JSON data. Using `VARIANT` type allows for flexible ingestion of semi-structured JSON.

In [ ]:
USE SCHEMA RAW;

CREATE OR REPLACE TRANSIENT TABLE CUSTOMER_INTERACTIONS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE PRODUCT_REVIEWS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE SUPPORT_TICKETS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE CUSTOMERS (data VARIANT);

## 6. Data Loading into Raw Tables
Copy data from the staged JSON files into the respective raw tables using the `COPY INTO` command and the previously defined JSON file format.

In [ ]:
USE SCHEMA RAW;

COPY INTO CUSTOMER_INTERACTIONS
FROM @RAW_DATA_STAGE/customer_interactions.json
FILE_FORMAT = JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO PRODUCT_REVIEWS
FROM @RAW_DATA_STAGE/product_reviews.json
FILE_FORMAT = JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO SUPPORT_TICKETS
FROM @RAW_DATA_STAGE/support_tickets.json
FILE_FORMAT = JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO CUSTOMERS
FROM @RAW_DATA_STAGE/customers.json
FILE_FORMAT = JSON_FORMAT
ON_ERROR = 'CONTINUE';

## 7. Staging Layer - Data Cleaning and Transformation
Transform the raw JSON data into structured staging tables in the `ANALYTICS` schema. This involves selecting specific fields, casting data types, and handling potential errors during conversion (e.g., using `TRY_TO_TIMESTAMP_NTZ`).

In [ ]:
USE SCHEMA ANALYTICS;

-- Customer Interactions Staging Table
CREATE OR REPLACE TABLE STG_CUSTOMER_INTERACTIONS AS
SELECT
    data:interaction_id::VARCHAR AS interaction_id,
    data:customer_id::VARCHAR AS customer_id,
    TRY_TO_TIMESTAMP_NTZ(data:interaction_date::VARCHAR) AS interaction_date,
    data:agent_id::VARCHAR AS agent_id,
    data:interaction_type::VARCHAR AS interaction_type,
    data:interaction_notes::VARCHAR AS interaction_notes
FROM RAW.CUSTOMER_INTERACTIONS;

-- Product Reviews Staging Table
CREATE OR REPLACE TABLE STG_PRODUCT_REVIEWS AS
SELECT
    data:review_id::VARCHAR AS review_id,
    data:customer_id::VARCHAR AS customer_id,
    data:product_id::VARCHAR AS product_id,
    TRY_TO_TIMESTAMP_NTZ(data:review_date::VARCHAR) AS review_date,
    data:review_rating::NUMBER AS review_rating,
    data:review_text::VARCHAR AS review_text,
    data:review_language::VARCHAR AS review_language
FROM RAW.PRODUCT_REVIEWS;

-- Support Tickets Staging Table
CREATE OR REPLACE TABLE STG_SUPPORT_TICKETS AS
SELECT
    data:ticket_id::VARCHAR AS ticket_id,
    data:customer_id::VARCHAR AS customer_id,
    TRY_TO_TIMESTAMP_NTZ(data:ticket_date::VARCHAR) AS ticket_date,
    data:ticket_status::VARCHAR AS ticket_status,
    data:ticket_category::VARCHAR AS ticket_category,
    data:ticket_description::VARCHAR AS ticket_description
FROM RAW.SUPPORT_TICKETS;

## 8. Customer Dimension Table
Create a customer dimension table (`CUSTOMER_BASE`) from the raw customers data. This table holds key customer attributes.

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE CUSTOMER_BASE AS
SELECT
    data:customer_id::VARCHAR AS customer_id,
    data:persona::VARCHAR AS persona,
    TRY_TO_DATE(data:sign_up_date::VARCHAR) AS sign_up_date,
    data:products_owned::NUMBER AS products_owned,
    data:lifetime_value::NUMBER AS lifetime_value
FROM RAW.CUSTOMERS;

## 9. Fact Tables - Enriched with Cortex LLM Functions
Create fact tables from the staging tables. These tables are enriched with insights derived from Snowflake Cortex LLM functions:
- `SNOWFLAKE.CORTEX.SENTIMENT` for sentiment analysis.
- `SNOWFLAKE.CORTEX.TRANSLATE` for standardizing text to English.
- `SNOWFLAKE.CORTEX.CLASSIFY_TEXT` for categorizing support ticket priority.
- `SNOWFLAKE.CORTEX.COMPLETE` for extracting specific information like expected resolution times and requested remedies.

### 9.1 Fact Customer Interactions

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE FACT_CUSTOMER_INTERACTIONS AS
SELECT
    i.interaction_id,
    i.customer_id,
    i.interaction_date,
    i.agent_id,
    i.interaction_type,
    i.interaction_notes,
    SNOWFLAKE.CORTEX.SENTIMENT(i.interaction_notes) AS sentiment_score
FROM ANALYTICS.STG_CUSTOMER_INTERACTIONS i
WHERE i.interaction_notes IS NOT NULL;

### 9.2 Fact Product Reviews

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE FACT_PRODUCT_REVIEWS AS
SELECT
    r.review_id,
    r.customer_id,
    r.product_id,
    r.review_date,
    r.review_rating,
    r.review_text,
    r.review_language,
    SNOWFLAKE.CORTEX.SENTIMENT(r.review_text) AS sentiment_score,
    CASE
        WHEN CONTAINS(LOWER(r.review_language), 'german') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'de', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'french') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'fr', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'spanish') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'es', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'italian') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'it', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'portuguese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'pt', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'chinese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'zh', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'japanese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ja', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'korean') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ko', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'russian') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ru', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'arabic') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ar', 'en')
        ELSE r.review_text
    END AS review_text_english
FROM ANALYTICS.STG_PRODUCT_REVIEWS r
WHERE r.review_text IS NOT NULL;

### 9.3 Fact Support Tickets

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE FACT_SUPPORT_TICKETS AS
SELECT
    t.ticket_id,
    t.customer_id,
    t.ticket_date,
    t.ticket_status,
    t.ticket_category,
    t.ticket_description,
    SNOWFLAKE.CORTEX.SENTIMENT(t.ticket_description) AS sentiment_score,
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        t.ticket_description,
        [
            {
                'label': 'Critical',
                'description': 'Requires immediate attention and resolution',
                'examples': [
                    'System is down and customers cannot place orders',
                    'Security breach detected',
                    'Payment processing completely stopped',
                    'All users locked out of the system',
                    'Cannot access my account',
                    'Order stuck in processing',
                    'Payment failed',
                    'Website not loading',
                    'Critical feature not working',
                    'Data loss or corruption'
                ]
            },
            {
                'label': 'High',
                'description': 'Should be resolved within 24 hours',
                'examples': [
                    'Customer cannot complete checkout',
                    'Order status stuck in processing',
                    'Unable to access account',
                    'Payment failed for multiple customers',
                    'Product not working as expected',
                    'Account access issues',
                    'Billing problems',
                    'Service disruption',
                    'Performance issues',
                    'Security concerns'
                ]
            },
            {
                'label': 'Medium',
                'description': 'Should be resolved within 3 days',
                'examples': [
                    'Product image not displaying correctly',
                    'Slow response times in certain areas',
                    'Minor UI issues',
                    'Account settings not saving',
                    'Feature not working as expected',
                    'Minor display issues',
                    'Non-critical performance problems',
                    'General functionality questions',
                    'Minor account issues',
                    'Non-urgent technical problems'
                ]
            },
            {
                'label': 'Low',
                'description': 'Can be handled in regular queue',
                'examples': [
                    'General product questions',
                    'Feature request',
                    'UI enhancement suggestion',
                    'Documentation clarification',
                    'How-to questions',
                    'Product information request',
                    'General feedback',
                    'Non-technical questions',
                    'Account information request',
                    'General support questions'
                ]
            }
        ],
        {
            'task_description': 'Classify the urgency level of this support ticket based on its description. When in doubt, classify as Critical or High priority if the issue affects core functionality or user access.'
        }
    )['label'] AS priority_level,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-4-sonnet', -- Using a placeholder model, ensure this is updated if a specific model is required/available
        'What specific timeframe or deadline does the customer mention or expect for resolution? ' || t.ticket_description
    ) AS expected_resolution_timeframe,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-4-sonnet', -- Using a placeholder model
        'What compensation, refund, or specific remedy is the customer seeking? ' || t.ticket_description
    ) AS requested_remedy
FROM ANALYTICS.STG_SUPPORT_TICKETS t
WHERE t.ticket_description IS NOT NULL;

## 10. Analysis Layer Tables
Create tables that aggregate and analyze data from the fact tables. This includes sentiment analysis over time, trends, and ticket patterns.

### 10.1 Sentiment Analysis (Combined)

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE SENTIMENT_ANALYSIS AS
WITH interaction_sentiment AS (
    SELECT
        customer_id,
        interaction_date,
        sentiment_score,
        'interaction' AS source_type
    FROM ANALYTICS.FACT_CUSTOMER_INTERACTIONS
),
review_sentiment AS (
    SELECT
        customer_id,
        review_date AS interaction_date,
        sentiment_score,
        'review' AS source_type
    FROM ANALYTICS.FACT_PRODUCT_REVIEWS
),
ticket_sentiment AS (
    SELECT
        customer_id,
        ticket_date AS interaction_date,
        sentiment_score,
        'ticket' AS source_type
    FROM ANALYTICS.FACT_SUPPORT_TICKETS
)
SELECT * FROM interaction_sentiment
UNION ALL
SELECT * FROM review_sentiment
UNION ALL
SELECT * FROM ticket_sentiment;

### 10.2 Sentiment Trends Analysis

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE SENTIMENT_TRENDS AS
WITH sentiment_data AS (
    SELECT
        customer_id,
        sentiment_score,
        interaction_date,
        FIRST_VALUE(sentiment_score) OVER (PARTITION BY customer_id ORDER BY interaction_date) AS first_sentiment,
        LAST_VALUE(sentiment_score) OVER (PARTITION BY customer_id ORDER BY interaction_date 
                                         ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_sentiment
    FROM ANALYTICS.SENTIMENT_ANALYSIS
)
SELECT
    customer_id,
    ARRAY_AGG(sentiment_score) WITHIN GROUP (ORDER BY interaction_date) AS sentiment_history,
    AVG(sentiment_score) AS avg_sentiment,
    MIN(sentiment_score) AS min_sentiment,
    MAX(sentiment_score) AS max_sentiment,
    MAX(sentiment_score) - MIN(sentiment_score) AS sentiment_volatility,
    CASE
        WHEN COUNT(*) > 1 THEN 
            MAX(last_sentiment) - MAX(first_sentiment) -- This will be the same as last_sentiment - first_sentiment due to MAX on already partitioned values
        ELSE 0
    END AS sentiment_trend
FROM sentiment_data
GROUP BY customer_id;

### 10.3 Ticket Pattern Analysis

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE TICKET_PATTERNS AS
SELECT
    customer_id,
    COUNT(*) AS ticket_count,
    MIN(ticket_date) AS first_ticket_date,
    MAX(ticket_date) AS most_recent_ticket_date,
    ARRAY_AGG(ticket_category) WITHIN GROUP (ORDER BY ticket_date) AS ticket_categories,
    ARRAY_AGG(priority_level) WITHIN GROUP (ORDER BY ticket_date) AS ticket_priorities
FROM ANALYTICS.FACT_SUPPORT_TICKETS
GROUP BY customer_id;

### 10.4 Customer Insight Summaries (using Cortex LLM)

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE INSIGHT_SUMMARIES AS
SELECT
    customer_id,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-4-sonnet', -- Using a placeholder model
        [
            {
                'role': 'user',
                'content': 'Summarize the following customer interactions in 100 words or less: ' || 
                ARRAY_TO_STRING(
                    ARRAY_AGG(
                        CASE
                            WHEN interaction_notes IS NOT NULL THEN interaction_notes
                            WHEN review_text IS NOT NULL THEN review_text
                            WHEN ticket_description IS NOT NULL THEN ticket_description
                        END
                    ),
                    ' | '
                )
            }
        ],
        {'max_tokens': 100}
    ) AS customer_summary
FROM (
    SELECT customer_id, interaction_notes, NULL as review_text, NULL as ticket_description
    FROM ANALYTICS.FACT_CUSTOMER_INTERACTIONS
    UNION ALL
    SELECT customer_id, NULL, review_text, NULL
    FROM ANALYTICS.FACT_PRODUCT_REVIEWS
    UNION ALL
    SELECT customer_id, NULL, NULL, ticket_description
    FROM ANALYTICS.FACT_SUPPORT_TICKETS
)
GROUP BY customer_id;

## 11. Customer Persona Analysis Table
Combines data from various analytical tables (`CUSTOMER_BASE`, `SENTIMENT_TRENDS`, `INSIGHT_SUMMARIES`, `TICKET_PATTERNS`, `FACT_PRODUCT_REVIEWS`) to create a comprehensive `CUSTOMER_PERSONA_SIGNALS` table. This table includes derived personas, churn risk, and upsell opportunities based on customer behavior and sentiment.

In [ ]:
USE SCHEMA ANALYTICS;

CREATE OR REPLACE TABLE CUSTOMER_PERSONA_SIGNALS AS
SELECT 
    cb.customer_id,
    st.avg_sentiment,
    st.sentiment_trend,
    st.sentiment_volatility,
    CASE 
        WHEN st.avg_sentiment < -0.3 THEN 'Negative'
        WHEN st.avg_sentiment > 0.3 THEN 'Positive'
        ELSE 'Neutral'
    END AS overall_sentiment,
    tpat.ticket_count,
    tpat.ticket_categories,
    tpat.ticket_priorities,
    AVG(pr.review_rating) AS avg_rating, -- Aggregating review rating
    is_summary.customer_summary,
    CASE
        WHEN st.avg_sentiment > 0.5 AND COALESCE(tpat.ticket_count, 0) <= 1 THEN 'Satisfied'
        WHEN st.avg_sentiment < -0.3 AND COALESCE(tpat.ticket_count, 0) >= 3 THEN 'Frustrated'
        WHEN st.sentiment_volatility > 0.7 THEN 'Mixed'
        WHEN st.sentiment_trend > 0.3 THEN 'Improving'
        WHEN st.sentiment_trend < -0.3 THEN 'Deteriorating'
        ELSE 'Neutral'
    END AS derived_persona,
    CASE
        WHEN (st.avg_sentiment < -0.3 AND COALESCE(tpat.ticket_count, 0) >= 1) 
             OR (st.sentiment_trend < -0.2 AND COALESCE(tpat.ticket_count, 0) >= 1)
             OR (st.avg_sentiment < -0.2 AND st.sentiment_trend < -0.1) THEN 'High'
        WHEN (st.avg_sentiment < -0.1 AND st.sentiment_trend < 0) 
             OR (COALESCE(tpat.ticket_count, 0) >= 2) THEN 'Medium'
        ELSE 'Low'
    END AS churn_risk,
    CASE
        WHEN st.avg_sentiment > 0.3 AND COALESCE(tpat.ticket_count, 0) <= 1 THEN 'High'
        WHEN st.sentiment_trend > 0.3 THEN 'Medium'
        ELSE 'Low'
    END AS upsell_opportunity
FROM ANALYTICS.CUSTOMER_BASE cb
LEFT JOIN ANALYTICS.SENTIMENT_TRENDS st USING (customer_id)
LEFT JOIN ANALYTICS.INSIGHT_SUMMARIES is_summary USING (customer_id)
LEFT JOIN ANALYTICS.TICKET_PATTERNS tpat USING (customer_id)
LEFT JOIN ANALYTICS.FACT_PRODUCT_REVIEWS pr USING (customer_id)
GROUP BY 
    cb.customer_id, 
    st.avg_sentiment, 
    st.sentiment_trend, 
    st.sentiment_volatility,
    tpat.ticket_count,
    tpat.ticket_categories,
    tpat.ticket_priorities,
    is_summary.customer_summary; -- Removed cb.persona as it's not used in SELECT or CASE statements and ensures proper grouping

## 12. 🎯 Time to Get Some Sweet Insights! 
Run analytical queries against the `CUSTOMER_PERSONA_SIGNALS` and other fact/analytical tables to derive actionable insights.

### 12.1 📊 Customer Vibes Overview - How's everyone feeling?

In [ ]:
SELECT 
    overall_sentiment,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment_score,
    ROUND(AVG(avg_rating), 2) as avg_product_rating
FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS
GROUP BY overall_sentiment
ORDER BY customer_count DESC;

### 12.2 🚨 Churn Risk Check - Who's thinking about ghosting us?

In [ ]:
SELECT 
    churn_risk,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment_score,
    ROUND(AVG(sentiment_trend), 2) as avg_sentiment_trend,
    SUM(ticket_count) as total_tickets,
    ROUND(AVG(ticket_count), 1) as avg_tickets_per_customer,
    COUNT(CASE WHEN overall_sentiment = 'Negative' THEN 1 END) as negative_sentiment_count,
    COUNT(CASE WHEN overall_sentiment = 'Positive' THEN 1 END) as positive_sentiment_count
FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS
GROUP BY churn_risk
ORDER BY total_tickets DESC, avg_sentiment_score ASC
LIMIT 10;

### 12.3 🎫 Support Ticket Heat Map - What's hot in the support queue?

In [ ]:
SELECT 
    priority_level,
    COUNT(*) as ticket_count,
    ROUND(AVG(sentiment_score), 2) as avg_sentiment,
    COUNT(DISTINCT customer_id) as unique_customers
FROM ANALYTICS.FACT_SUPPORT_TICKETS
GROUP BY priority_level
ORDER BY 
    CASE priority_level
        WHEN 'Critical' THEN 1
        WHEN 'High' THEN 2
        WHEN 'Medium' THEN 3
        WHEN 'Low' THEN 4
    END;

### 12.4 🗺️ Customer Journey Map - Plot those customer adventures

In [ ]:
WITH customer_journey AS (
    SELECT 
        cps.customer_id,
        cps.derived_persona,
        cps.churn_risk,
        cps.upsell_opportunity,
        cps.avg_sentiment,
        cps.sentiment_trend,
        cps.ticket_count,
        cps.avg_rating,
        cps.customer_summary
    FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS cps
)
SELECT 
    derived_persona,
    churn_risk,
    upsell_opportunity,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment,
    ROUND(AVG(avg_rating), 2) as avg_rating,
    ROUND(AVG(ticket_count), 2) as avg_tickets
FROM customer_journey
GROUP BY derived_persona, churn_risk, upsell_opportunity
ORDER BY customer_count DESC;

### 12.5 🌍 Global Customer Vibes - How's the love worldwide?

In [ ]:
SELECT 
    review_language,
    COUNT(*) as review_count,
    ROUND(AVG(review_rating), 2) as avg_rating,
    ROUND(AVG(sentiment_score), 2) as avg_sentiment,
    COUNT(DISTINCT customer_id) as unique_customers
FROM ANALYTICS.FACT_PRODUCT_REVIEWS
GROUP BY review_language
ORDER BY review_count DESC;

### 12.6 🤝 Customer Touch Points - Where and how are they reaching out?

In [ ]:
SELECT 
    interaction_type,
    COUNT(*) as interaction_count,
    ROUND(AVG(sentiment_score), 2) as avg_sentiment,
    COUNT(DISTINCT customer_id) as unique_customers
FROM ANALYTICS.FACT_CUSTOMER_INTERACTIONS
GROUP BY interaction_type
ORDER BY interaction_count DESC;

### 12.7 💰 Value vs. Vibes Analysis - Are our VIPs feeling the love?

In [ ]:
SELECT 
    CASE 
        WHEN lifetime_value >= 1000 THEN 'High Value'
        WHEN lifetime_value >= 500 THEN 'Medium Value'
        ELSE 'Low Value'
    END as value_segment,
    COUNT(*) as customer_count,
    ROUND(AVG(cps.avg_sentiment), 2) as avg_sentiment, -- aliased avg_sentiment to avoid ambiguity
    ROUND(AVG(cps.ticket_count), 2) as avg_tickets,   -- aliased ticket_count
    ROUND(AVG(cps.avg_rating), 2) as avg_rating     -- aliased avg_rating
FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS cps
JOIN ANALYTICS.CUSTOMER_BASE cb USING (customer_id)
GROUP BY value_segment
ORDER BY 
    CASE value_segment
        WHEN 'High Value' THEN 1
        WHEN 'Medium Value' THEN 2
        WHEN 'Low Value' THEN 3
    END;

## 13. Table and Column Comments
Add descriptive comments to tables and columns to improve data discoverability and understanding.

In [ ]:
USE SCHEMA ANALYTICS;

-- Table comments
COMMENT ON TABLE CUSTOMER_BASE IS 'Core customer information containing demographic data and customer identifiers';
COMMENT ON TABLE FACT_CUSTOMER_INTERACTIONS IS 'Customer interaction events with timestamps, interaction types, and sentiment analysis scores';
COMMENT ON TABLE FACT_PRODUCT_REVIEWS IS 'Product reviews with ratings, review text, and sentiment analysis across multiple languages';
COMMENT ON TABLE FACT_SUPPORT_TICKETS IS 'Support tickets with priority levels, resolution times, and customer satisfaction metrics';
COMMENT ON TABLE SENTIMENT_TRENDS IS 'Aggregated customer sentiment metrics including trends, volatility, and average sentiment scores';
COMMENT ON TABLE INSIGHT_SUMMARIES IS 'Cortex LLM function summaries of customer behavior and preferences for business insights';
COMMENT ON TABLE TICKET_PATTERNS IS 'Patterns and trends in customer support tickets including categorization and priority distribution';
COMMENT ON TABLE CUSTOMER_PERSONA_SIGNALS IS 'Customer segmentation data with derived personas, churn risk, and upsell opportunity indicators';

-- Column comments for STG_CUSTOMER_INTERACTIONS
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.interaction_id IS 'Unique identifier for each customer interaction';
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.interaction_date IS 'Timestamp when the interaction occurred';
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.agent_id IS 'Unique identifier for the customer service agent';
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.interaction_type IS 'Type of customer interaction (e.g., call, email, chat)';
COMMENT ON COLUMN STG_CUSTOMER_INTERACTIONS.interaction_notes IS 'Detailed notes or transcript of the interaction';

-- Column comments for STG_PRODUCT_REVIEWS
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.review_id IS 'Unique identifier for each product review';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.customer_id IS 'Unique identifier for the customer who wrote the review';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.product_id IS 'Unique identifier for the reviewed product';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.review_date IS 'Timestamp when the review was posted';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.review_rating IS 'Numeric rating given by the customer (1-5)';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.review_text IS 'Text content of the product review';
COMMENT ON COLUMN STG_PRODUCT_REVIEWS.review_language IS 'Language code of the review text';

-- Column comments for STG_SUPPORT_TICKETS
COMMENT ON COLUMN STG_SUPPORT_TICKETS.ticket_id IS 'Unique identifier for each support ticket';
COMMENT ON COLUMN STG_SUPPORT_TICKETS.customer_id IS 'Unique identifier for the customer who created the ticket';
COMMENT ON COLUMN STG_SUPPORT_TICKETS.ticket_date IS 'Timestamp when the ticket was created';
COMMENT ON COLUMN STG_SUPPORT_TICKETS.ticket_status IS 'Current status of the support ticket';
COMMENT ON COLUMN STG_SUPPORT_TICKETS.ticket_category IS 'Category classification of the support ticket';
COMMENT ON COLUMN STG_SUPPORT_TICKETS.ticket_description IS 'Detailed description of the support issue';

-- Column comments for CUSTOMER_BASE
COMMENT ON COLUMN CUSTOMER_BASE.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN CUSTOMER_BASE.persona IS 'Customer persona classification';
COMMENT ON COLUMN CUSTOMER_BASE.sign_up_date IS 'Date when the customer first registered';
COMMENT ON COLUMN CUSTOMER_BASE.products_owned IS 'Number of products owned by the customer';
COMMENT ON COLUMN CUSTOMER_BASE.lifetime_value IS 'Total lifetime value of the customer';

-- Column comments for FACT_CUSTOMER_INTERACTIONS
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.interaction_id IS 'Unique identifier for each customer interaction';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.interaction_date IS 'Timestamp when the interaction occurred';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.agent_id IS 'Unique identifier for the customer service agent';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.interaction_type IS 'Type of customer interaction (e.g., call, email, chat)';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.interaction_notes IS 'Detailed notes or transcript of the interaction';
COMMENT ON COLUMN FACT_CUSTOMER_INTERACTIONS.sentiment_score IS 'Cortex LLM function sentiment score for the interaction (-1 to 1)';

-- Column comments for FACT_PRODUCT_REVIEWS
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_id IS 'Unique identifier for each product review';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.customer_id IS 'Unique identifier for the customer who wrote the review';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.product_id IS 'Unique identifier for the reviewed product';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_date IS 'Timestamp when the review was posted';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_rating IS 'Numeric rating given by the customer (1-5)';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_text IS 'Text content of the product review';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_language IS 'Language code of the review text';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.sentiment_score IS 'Cortex LLM function sentiment score for the review (-1 to 1)';
COMMENT ON COLUMN FACT_PRODUCT_REVIEWS.review_text_english IS 'English translation of the review text';

-- Column comments for FACT_SUPPORT_TICKETS
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.ticket_id IS 'Unique identifier for each support ticket';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.customer_id IS 'Unique identifier for the customer who created the ticket';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.ticket_date IS 'Timestamp when the ticket was created';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.ticket_status IS 'Current status of the support ticket';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.ticket_category IS 'Category classification of the support ticket';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.ticket_description IS 'Detailed description of the support issue';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.sentiment_score IS 'Cortex LLM function sentiment score for the ticket (-1 to 1)';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.priority_level IS 'Cortex-classified priority level (Critical, High, Medium, Low)';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.expected_resolution_timeframe IS 'Cortex-extracted customer expected resolution timeframe';
COMMENT ON COLUMN FACT_SUPPORT_TICKETS.requested_remedy IS 'Cortex-extracted customer requested compensation or remedy';

-- Column comments for SENTIMENT_ANALYSIS
COMMENT ON COLUMN SENTIMENT_ANALYSIS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN SENTIMENT_ANALYSIS.interaction_date IS 'Timestamp of the interaction';
COMMENT ON COLUMN SENTIMENT_ANALYSIS.sentiment_score IS 'Cortex LLM function sentiment score (-1 to 1)';
COMMENT ON COLUMN SENTIMENT_ANALYSIS.source_type IS 'Type of interaction (interaction, review, ticket)';

-- Column comments for SENTIMENT_TRENDS
COMMENT ON COLUMN SENTIMENT_TRENDS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN SENTIMENT_TRENDS.sentiment_history IS 'Array of sentiment scores over time';
COMMENT ON COLUMN SENTIMENT_TRENDS.avg_sentiment IS 'Average sentiment score across all interactions';
COMMENT ON COLUMN SENTIMENT_TRENDS.min_sentiment IS 'Lowest sentiment score recorded';
COMMENT ON COLUMN SENTIMENT_TRENDS.max_sentiment IS 'Highest sentiment score recorded';
COMMENT ON COLUMN SENTIMENT_TRENDS.sentiment_volatility IS 'Range between min and max sentiment scores';
COMMENT ON COLUMN SENTIMENT_TRENDS.sentiment_trend IS 'Change in sentiment from first to last interaction';

-- Column comments for TICKET_PATTERNS
COMMENT ON COLUMN TICKET_PATTERNS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN TICKET_PATTERNS.ticket_count IS 'Total number of support tickets';
COMMENT ON COLUMN TICKET_PATTERNS.first_ticket_date IS 'Date of first support ticket';
COMMENT ON COLUMN TICKET_PATTERNS.most_recent_ticket_date IS 'Date of most recent support ticket';
COMMENT ON COLUMN TICKET_PATTERNS.ticket_categories IS 'Array of ticket categories in chronological order';
COMMENT ON COLUMN TICKET_PATTERNS.ticket_priorities IS 'Array of ticket priorities in chronological order';

-- Column comments for INSIGHT_SUMMARIES
COMMENT ON COLUMN INSIGHT_SUMMARIES.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN INSIGHT_SUMMARIES.customer_summary IS 'Cortex LLM function summary of customer interactions';

-- Column comments for CUSTOMER_PERSONA_SIGNALS
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.customer_id IS 'Unique identifier for the customer';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.avg_sentiment IS 'Average sentiment score across all interactions';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.sentiment_trend IS 'Overall trend in sentiment over time';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.sentiment_volatility IS 'Variability in sentiment scores';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.overall_sentiment IS 'Categorized sentiment (Positive, Neutral, Negative)';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.ticket_count IS 'Total number of support tickets';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.ticket_categories IS 'Array of ticket categories';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.ticket_priorities IS 'Array of ticket priorities';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.avg_rating IS 'Average product review rating';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.customer_summary IS 'Cortex LLM function summary of customer interactions';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.derived_persona IS 'Cortex-classified customer persona type';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.churn_risk IS 'Predicted risk of customer churn (High, Medium, Low)';
COMMENT ON COLUMN CUSTOMER_PERSONA_SIGNALS.upsell_opportunity IS 'Predicted opportunity for upselling (High, Medium, Low)';

## 14. 🎉 Final Setup - Stage for Semantic Models
Create a stage in the `SEMANTIC_MODELS` schema, possibly for exporting or versioning semantic model definitions or outputs.

In [ ]:
USE SCHEMA SEMANTIC_MODELS;

CREATE OR REPLACE STAGE CUSTOMER_INSIGHTS;

## End of Notebook
All SQL commands from the provided script have been organized into this Snowflake Notebook structure.